# Monitoramento de modelos de machine learning

Cliente que consome da API para avaliação de performance e aderência

*Obs. Antes de rodar o cliente, inicie o servidor `monitoring\app\main.py`*

In [109]:
import json
import requests

## Performance e volumetria
Obtendo a performance do modelo para uma dada lista de registros, obtendo
 - A volumetria (quantidade de registros) para cada mês presente na lista de registros
 - O valor da área sob a curva ROC do modelo pré-treinado "../monitoring/model.pkl" no dado conjunto de registros.

In [110]:
def calc_performance(batch_records):
    performance_url = 'http://127.0.0.1:8001/v1/performance/'

    performance_res = requests.post(performance_url, json = batch_records)
    print("Resposta da Requisição: ", performance_res.text)
    return json.loads(performance_res.text)


In [111]:
batch_records = json.load(open("batch_records.json"))
perform_json = calc_performance(batch_records)

Resposta da Requisição:  {"Volumetry":{"3-2017":62,"8-2017":72,"1-2017":58,"4-2017":49,"6-2017":63,"7-2017":74,"5-2017":67,"2-2017":55},"Performance":0.5751748251748252}


Resultado da performance do modelo para a lista de registros em "batch_records.json"

In [112]:
print("------------------------")
print("Volumetria: ", sorted(perform_json['Volumetry'].items()))
print("Performance (Área sob a curva ROC): ", perform_json['Performance'])

------------------------
Volumetria:  [('1-2017', 58), ('2-2017', 55), ('3-2017', 62), ('4-2017', 49), ('5-2017', 67), ('6-2017', 63), ('7-2017', 74), ('8-2017', 72)]
Performance (Área sob a curva ROC):  0.5751748251748252


## Aderência
Obtendo a aderência do modelo utilizando a distância distribuição de scores de uma dada base para a da distribuição vista na base de Teste.
Utilizando o Kolmogorov–Smirnov test para calcular a distância

In [113]:
def calc_adherence(dataset_path):
    adherence_url = 'http://127.0.0.1:8001/v1/aderencia/'
    
    adherence_res = requests.post(adherence_url, json = {'dataset_path': dataset_path})
    print("Resposta da Requisição: ", adherence_res.text)
    if (adherence_res.status_code == 200):
        return json.loads(adherence_res.text)
    else:
        return None

### Aderência do modelo - base de treino x teste

In [114]:
train_dataset_path =  "datasets/credit_01/train.gz"
adherence_json = calc_adherence(train_dataset_path)

Resposta da Requisição:  {"Adherence":[0.002759858953621075,0.9605978662359891]}


Resultado da distância distribuição de scores da base de treino para a da distribuição vista na base de Teste

In [115]:
print("------------------------")
print("Distância da distribuição vista na base de Teste: ", adherence_json['Adherence'][0])

------------------------
Distância da distribuição vista na base de Teste:  0.002759858953621075


### Aderência do modelo - base de out of time x teste

In [116]:
oot_dataset_path =  "datasets\credit_01\oot.gz"
adherence_json = calc_adherence(oot_dataset_path)

Resposta da Requisição:  Internal Server Error
